In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
tf.ones([1, 3], tf.int32)

<tf.Tensor: id=2, shape=(1, 3), dtype=int32, numpy=array([[1, 1, 1]], dtype=int32)>

## Old stuff

In [3]:
class GetEmbeddings(tf.keras.layers.Layer):
    def __init__(self, num_buckets=None, embedding_size=None,*args, **kwargs):
        super(GetEmbeddings, self).__init__(*args, **kwargs)

    def build(self, input_shape):
        self.embedding = tf.constant([[1.0,1.0,1.0,1.0],[2.0,2.0,2.0,2.0],[3.0,3.0,3.0,3.0],[4.0,4.0,4.0,4.0]]) # Embedding matrix
    
    def call(self, input):
        user = input[0]
        movie = input[1]
        # Expand user to get it to a matrix of 1 and zeros instead
        user = tf.expand_dims(user, 2)
        user = tf.tile(user, [1,1, 4])
        # check equality of array in t1 for each array in t2 element by element. This is possible because the equal function supports broadcasting. 
        equal =  tf.math.equal(tf.constant([1.0,1.0,1.0,1.0]), user)
        # checking if all elements from  t1 match for all elements of some array in t2
        equal_all = tf.reduce_all(equal, axis=2)
        user_embeddings = self.embedding * user 
        # Get embeddings for the users based upon the movies he/she watched
        # Expand the movie first
        movies = tf.expand_dims(movie,2)
        movies = tf.tile(movies, [1,1, 4])
        # Get the embeddings
        movie_embeddings = self.embedding * movies
        # Only want the movies with values
        equal_movie =  tf.math.not_equal(tf.constant([0.0,0.0,0.0,0.0]), movie_embeddings)
        equal_all_movie = tf.reduce_all(equal, axis=1)
        # Get the indicies where we have a movie
        indices = tf.where(equal_movie)
        # In order to to the pointwise multiplication I need to have the same shape so expand
        out = tf.reshape(tf.gather_nd(movie_embeddings,indices),shape=(2,1,4))
        out = tf.tile(out,multiples=[1,4,1])
        # Point wise multiplication, each movie the user watched multiplied with this movie
        output = tf.math.multiply(out,user_embeddings)
        lenght =  tf.reduce_sum(tf.cast(equal_all, tf.float32),axis=1)
        mean = tf.reduce_sum(tf.cast(output,dtype=tf.float32),axis=1) / tf.reshape(lenght, (-1, 1))
        return mean

## Custom layer

In [4]:
class GetEmbeddings(tf.keras.layers.Layer):
    def __init__(self, num_buckets=None, embedding_size=None,*args, **kwargs):
        super(GetEmbeddings, self).__init__(*args, **kwargs)
        self.movies = num_buckets
        self.embedding_size = embedding_size
        #self.shape = compute_output_shape
    
    def build(self,input_shape):
        self.embedding = tf.constant([[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]]) # Embedding matrix
        self.kernel = self.add_variable("kernel",
                                    shape=[self.movies,
                                           self.embedding_size],dtype=tf.float32,initializer=tf.keras.initializers.RandomUniform(),trainable=True)
        
    def call(self, input):
        #input = tf.reshape(input)
        #movies_new = concated[:,:4]
        #users_new = concated[:,4:]
        user = input[:,:self.movies]
        movie = input[:,self.movies:]
        # Expand user to get it to a matrix of 1 and zeros instead
        user = tf.expand_dims(user, 2)
        user = tf.tile(user, [1,1, self.embedding_size])
        # check equality of array in t1 for each array in t2 element by element. This is possible because the equal function supports broadcasting. 
        #equal =  tf.math.equal(tf.constant([1,1,1,1]), user)       
        equal =  tf.math.equal(tf.ones([1, self.embedding_size]), user)
        # checking if all elements from  t1 match for all elements of some array in t2
        equal_all = tf.reduce_all(equal, axis=2)      
        #user_embeddings = self.embedding * user 
        user_embeddings = self.kernel * user
        # Get embeddings for the users based upon the movies he/she watched
        # Expand the movie first
        movies = tf.expand_dims(movie,2)
        movies = tf.tile(movies, [1,1, self.embedding_size])
        # Get the embeddings
        #movie_embeddings = self.embedding * movies      
        movie_embeddings = self.kernel * movies        
        # Only want the movies with values 
        equal_movie =  tf.math.not_equal(tf.zeros([1, self.embedding_size]), movie_embeddings)
        #print(equal_movie)
        equal_all_movie = tf.reduce_all(equal, axis=1)
        #print(equal_all_movie)
        # Get the indicies where we have a movie
        indices = tf.where(equal_movie)
        # In order to to the pointwise multiplication I need to have the same shape so expand
        #print(tf.gather_nd(movie_embeddings,indices))
        out = tf.reshape(tf.gather_nd(movie_embeddings,indices),shape=(tf.shape(user)[0],1,self.embedding_size))
        out = tf.tile(out,multiples=[1,self.movies,1])
        #print(tf.math.multiply(out[1],user_embeddings))
        # Point wise multiplication, each movie the user watched multiplied with this movie
        output = tf.math.multiply(user_embeddings,out)
        #print(output)
        #print(tf.shape(output))
        #output = tf.reshape(output,shape=(self.batch_size,self.movies,self.embedding_size))
        mean = tf.reduce_sum(tf.cast(output,dtype=tf.float32),axis=1) / self.movies
        return mean

In [5]:
layer = GetEmbeddings(4,10)
user = tf.constant([[1.0,1.0,1.0,1.0],[1.0,0.0,0.0,0.0],[1.0,0.0,0.0,0.0],[1.0,0.0,0.0,0.0]]) # Example user, two users
#PROBLEMS PROBLEMS 
movie = tf.constant([[1.0,0.0,0.0,0.0],[0.0,1.0,0.0,0.0],[0.0,0.0,1.0,0.0],[0.0,0.0,1.0,0.0]]) 
concated = tf.concat([tf.cast(user,dtype=tf.int32),tf.cast(movie,tf.int32)],axis=1)
concated = tf.cast(concated,tf.float32)
layer(concated)

<tf.Tensor: id=75, shape=(4, 10), dtype=float32, numpy=
array([[ 1.13413483e-03,  5.88357070e-05,  1.05595325e-04,
         2.26339384e-04, -1.56531125e-04, -6.48036876e-05,
        -1.94309861e-04, -1.63331410e-04, -3.64638167e-04,
        -9.99023905e-06],
       [ 4.24248428e-04,  2.68042670e-04, -2.01234405e-04,
         1.02813421e-04, -6.06444883e-05,  9.91362976e-05,
        -1.80924799e-05, -5.02739385e-05, -1.45078564e-04,
         3.37964084e-05],
       [ 4.22851561e-04, -1.67394246e-04,  1.40525080e-04,
         2.05030810e-04, -3.40822567e-07, -8.94158075e-05,
        -2.33745464e-04, -5.64438014e-05, -7.81728595e-05,
        -1.97265967e-06],
       [ 4.22851561e-04, -1.67394246e-04,  1.40525080e-04,
         2.05030810e-04, -3.40822567e-07, -8.94158075e-05,
        -2.33745464e-04, -5.64438014e-05, -7.81728595e-05,
        -1.97265967e-06]], dtype=float32)>

In [6]:
concated

<tf.Tensor: id=11, shape=(4, 8), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0.]], dtype=float32)>

## Custom layer with dense layer connected

In [99]:
inputs = tf.keras.Input(shape=(8,)) # Don't forget the comma
x  = GetEmbeddings(4,10)(inputs)
x = tf.keras.layers.Dense(32,activation='relu')(x)
outputs = tf.keras.layers.Dense(1,activation='sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
get_embeddings_1 (GetEmbeddi (None, 10)                40        
_________________________________________________________________
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 425
Trainable params: 425
Non-trainable params: 0
_________________________________________________________________


In [100]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),  # Optimizer
              # Loss function to minimize
              loss=tf.keras.losses.BinaryCrossentropy(),
              # List of metrics to monitor
              metrics=[tf.keras.metrics.BinaryAccuracy()])



In [26]:
x_train = concated
y_train = tf.constant([0,0,0,0])
x_val=x_train
y_val=y_train

In [27]:
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=50,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=(x_val, y_val))

Train on 4 samples, validate on 4 samples
Epoch 1/50
4/4 [==============================] - 0s 2ms/sample - loss: 0.2900 - binary_accuracy: 1.0000 - val_loss: 0.2805 - val_binary_accuracy: 1.0000
Epoch 2/50
4/4 [==============================] - 0s 2ms/sample - loss: 0.2805 - binary_accuracy: 1.0000 - val_loss: 0.2735 - val_binary_accuracy: 1.0000
Epoch 3/50
4/4 [==============================] - 0s 1ms/sample - loss: 0.2735 - binary_accuracy: 1.0000 - val_loss: 0.2676 - val_binary_accuracy: 1.0000
Epoch 4/50
4/4 [==============================] - 0s 1ms/sample - loss: 0.2676 - binary_accuracy: 1.0000 - val_loss: 0.2623 - val_binary_accuracy: 1.0000
Epoch 5/50
4/4 [==============================] - 0s 1ms/sample - loss: 0.2623 - binary_accuracy: 1.0000 - val_loss: 0.2575 - val_binary_accuracy: 1.0000
Epoch 6/50
4/4 [==============================] - 0s 853us/sample - loss: 0.2575 - binary_accuracy: 1.0000 - val_loss: 0.2529 - val_binary_accuracy: 1.0000
Epoch 7/50
4/4 [================

## Load the data

### Load the data form csv

In [121]:
import ast
df = pd.read_csv("data/1000test_Heynicklas.csv")

def flatten(input):
    """"""
    list_one = input[0]
    list_two = input[1]
    output = list_one + list_two
    return output

def list_converter(input):
    output = ast.literal_eval(input)
    output = [float(value) for value in output]
    return output

df['userID'] = df['userID'].apply(list_converter)
df['movieID'] = df['movieID'].apply(list_converter)


df['input']=df['userID']+df['movieID']

In [122]:
# len(list(df["userID"].values)[0])
# 3000

### Load the data to dataset

In [90]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('ratings')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [92]:
ds = df_to_dataset(df[["input","ratings"]], shuffle=True, batch_size=32)

### Run the model with the new data

In [123]:
test_df = df[["input","ratings"]].copy()

In [124]:
target = test_df.pop('ratings')
dict_slices = tf.data.Dataset.from_tensor_slices((test_df.to_dict('list'), target.values)).batch(1)

In [130]:
for _ in dict_slices.take(5):
    print(_)

({'input': <tf.Tensor: id=9467, shape=(1, 2000), dtype=float32, numpy=array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>}, <tf.Tensor: id=9468, shape=(1,), dtype=int64, numpy=array([1])>)
({'input': <tf.Tensor: id=9471, shape=(1, 2000), dtype=float32, numpy=array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>}, <tf.Tensor: id=9472, shape=(1,), dtype=int64, numpy=array([1])>)
({'input': <tf.Tensor: id=9475, shape=(1, 2000), dtype=float32, numpy=array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>}, <tf.Tensor: id=9476, shape=(1,), dtype=int64, numpy=array([1])>)
({'input': <tf.Tensor: id=9479, shape=(1, 2000), dtype=float32, numpy=array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>}, <tf.Tensor: id=9480, shape=(1,), dtype=int64, numpy=array([1])>)
({'input': <tf.Tensor: id=9483, shape=(1, 2000), dtype=float32, numpy=array([[1., 0., 0., ..., 0., 0., 0.]], dtype=float32)>}, <tf.Tensor: id=9484, shape=(1,), dtype=int64, numpy=array([1])>)


In [135]:
inputs = tf.keras.Input(shape=(2000,),name="input") # Don't forget the comma
x  = GetEmbeddings(1000,10)(inputs)
x = tf.keras.layers.Dense(32,activation='relu')(x)
outputs = tf.keras.layers.Dense(1,activation='sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [136]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),  # Optimizer
              # Loss function to minimize
              loss=tf.keras.losses.BinaryCrossentropy(),
              # List of metrics to monitor
              metrics=[tf.keras.metrics.BinaryAccuracy()])



In [137]:
model.fit(dict_slices,epochs=10)

W0628 11:59:47.377648 4573812160 training_utils.py:1353] Expected a shuffled dataset but input dataset `x` is not shuffled. Please invoke `shuffle()` on input dataset.


Epoch 1/10
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1055 - binary_accuracy: 0.9990
Epoch 2/10
1000/1000 [==============================] - 13s 13ms/step - loss: 1.4663e-08 - binary_accuracy: 1.0000
Epoch 3/10
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0000e+00 - binary_accuracy: 1.0000
Epoch 4/10
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0000e+00 - binary_accuracy: 1.0000
Epoch 5/10
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0000e+00 - binary_accuracy: 1.0000
Epoch 6/10
1000/1000 [==============================] - 15s 15ms/step - loss: 0.0000e+00 - binary_accuracy: 1.0000
Epoch 7/10
 507/1000 [==============>...............] - ETA: 7s - loss: 0.0000e+00 - binary_accuracy: 1.0000

KeyboardInterrupt: 